In [6]:
import numpy as np
import pandas as pd
from scipy.stats import kurtosis, skew
from scipy.signal import welch, find_peaks
import matplotlib.pyplot as plt
from scipy.fft import fftfreq, fft
from scipy import stats
%matplotlib tk

In [10]:
V_SF = pd.read_csv('MotorVRB_2025.csv')
V_2F = pd.read_csv('MotorV2F_2025.csv')

<center><h1>Time-Domain Features

In [7]:
def calculate_rms(signal):
    return np.sqrt(np.mean(signal**2))

def extract_signal_stats(signal, window_size=1680):
    start = 0
    stats_list = []
    epsilon = 1e-8  # Small value to avoid division by zero

    while start < len(signal):
        end = min(start + window_size, len(signal))
        cycle = signal[start:end]

        mean_val = np.mean(cycle)
        rms_val = calculate_rms(cycle)
        peak_val = np.max(np.abs(cycle))

        stats_list.append({
            'Kurtosis': kurtosis(cycle, fisher=True, bias=False),
            'Skewness': skew(cycle, bias=False),
            'Mean': mean_val,
            'Peak': peak_val,
            'Variance': np.var(cycle),
            'StandardDeviation': np.std(cycle),
            'RMS': rms_val,
            'ShapeFactor': rms_val / (np.abs(mean_val) + epsilon),
            'ImpulseFactor': peak_val / (np.abs(mean_val) + epsilon),
        })

        start += window_size  # Move to next window

    df_stats = pd.DataFrame(stats_list)
    return df_stats

<center><h1>Frequency-Domain Features

In [8]:
def extract_frequency_features(signal, Fs, window_size=1680):
    """
    Extracts frequency-domain features from a signal.

    Features:
    - Spectral Centroid
    - Spectral Bandwidth
    - Peak Frequency
    - Power Spectral Density (PSD) Total Power
    - Harmonic Content (2nd to 5th harmonics)
    - Total Harmonic Distortion (THD)

    Parameters:
    - signal: 1D numpy array of the signal
    - Fs: Sampling frequency in Hz
    - window_size: Number of samples per analysis window

    Returns:
    - Pandas DataFrame with features for each window
    """
    epsilon = 1e-8
    start = 0
    features = []

    while start < len(signal):
        end = min(start + window_size, len(signal))
        segment = signal[start:end]

        # Compute Power Spectral Density (PSD)
        freqs, psd = welch(segment, fs=Fs, nperseg=min(256, len(segment)))

        # Spectral Centroid
        spectral_centroid = np.sum(freqs * psd) / (np.sum(psd) + epsilon)

        # Spectral Bandwidth
        spectral_bandwidth = np.sqrt(np.sum(((freqs - spectral_centroid)**2) * psd) / (np.sum(psd) + epsilon))

        # Peak Frequency (frequency with max power)
        peak_freq = freqs[np.argmax(psd)]

        # Total Power in PSD (optional for reference)
        total_power = np.sum(psd) + epsilon

        # Harmonic Content (energy at 2nd to 5th harmonics of fundamental)
        peaks, _ = find_peaks(psd, height=np.max(psd) * 0.1)
        if len(peaks) >= 1:
            fundamental_freq = freqs[peaks[0]]
            harmonics = fundamental_freq * np.arange(2, 6)  # 2nd to 5th harmonics
            harmonic_energy = 0
            for h_freq in harmonics:
                idx = (np.abs(freqs - h_freq)).argmin()
                harmonic_energy += psd[idx]
            harmonic_content = harmonic_energy / total_power
        else:
            harmonic_content = 0

        # Total Harmonic Distortion (THD)
        if len(peaks) >= 1:
            fundamental_power = psd[peaks[0]] + epsilon
            total_harmonic_power = np.sum(psd[peaks[1:]])  # Power in higher peaks
            thd = total_harmonic_power / fundamental_power
        else:
            thd = 0

        features.append({
            'SpectralCentroid': spectral_centroid,
            'SpectralBandwidth': spectral_bandwidth,
            'PeakFrequency': peak_freq,
            'TotalPower': total_power,
            'HarmonicContent': harmonic_content,
            'THD': thd
        })

        start += window_size  # Next window

    return pd.DataFrame(features)

In [17]:
time_domain = extract_signal_stats(V_SF['Sinal-C80-Filter'])
freq_domain = extract_frequency_features(V_SF['Sinal-C80-Filter'], 50000)

In [18]:
time_domain

,Kurtosis,Skewness,Mean,Peak,Variance,StandardDeviation,RMS,ShapeFactor,ImpulseFactor
0,-0.404515,-0.201482,-0.000363,0.122560,0.002133,0.046182,0.046183,127.169837,337.483253
1,0.174395,-0.286812,-0.000022,0.125755,0.001748,0.041804,0.041804,1867.639616,5618.251364
2,-0.342889,-0.046741,-0.000118,0.128321,0.002504,0.050038,0.050038,423.617442,1086.347976
3,-0.274917,-0.050986,0.000056,0.130258,0.002720,0.052153,0.052153,929.531912,2321.612582
4,-0.151054,-0.120016,-0.000433,0.146860,0.003006,0.054827,0.054829,126.764646,339.542787
...,...,...,...,...,...,...,...,...,...
1543,-0.010175,-0.261311,0.000224,0.176784,0.002965,0.054452,0.054453,242.557758,787.479163
1544,0.195334,-0.183999,-0.000034,0.186287,0.003345,0.057838,0.057838,1710.411722,5508.939637
1545,0.044049,-0.205884,-0.000053,0.225744,0.004408,0.066390,0.066390,1263.670645,4296.839595
1546,0.904392,-0.181464,-0.000228,0.229797,0.004667,0.068312,0.068313,299.345976,1006.968798


In [19]:
freq_domain

,SpectralCentroid,SpectralBandwidth,PeakFrequency,TotalPower,HarmonicContent,THD
0,530.589102,303.135407,585.9375,0.000011,0.002926,0.0
1,509.142781,334.908363,585.9375,0.000010,0.005838,0.0
2,497.135554,279.000381,585.9375,0.000014,0.002391,0.0
3,497.544702,267.993075,585.9375,0.000015,0.002851,0.0
4,475.160604,260.163842,390.6250,0.000016,0.043182,0.0
...,...,...,...,...,...,...
1543,515.896483,280.659661,585.9375,0.000014,0.003702,0.0
1544,517.557488,268.425602,585.9375,0.000015,0.003680,0.0
1545,511.461648,241.610230,585.9375,0.000021,0.002795,0.0
1546,510.781006,228.077675,585.9375,0.000023,0.002741,0.0


In [20]:
time_domain = extract_signal_stats(V_2F['Sinal-C80-Filter'])
freq_domain = extract_frequency_features(V_2F['Sinal-C80-Filter'], 50000)

In [21]:
time_domain

,Kurtosis,Skewness,Mean,Peak,Variance,StandardDeviation,RMS,ShapeFactor,ImpulseFactor
0,-0.318288,-0.514135,0.000538,0.245771,0.007959,0.089213,0.089215,165.936358,457.124236
1,0.625454,-0.669031,-0.000908,0.302138,0.007798,0.088306,0.088310,97.297919,332.887449
2,0.527250,-0.641796,-0.000261,0.235275,0.005005,0.070743,0.070744,271.421849,902.674885
3,0.412579,-0.691673,0.000197,0.258666,0.006090,0.078038,0.078038,396.946459,1315.726741
4,0.091687,-0.633761,-0.000122,0.160309,0.003377,0.058111,0.058111,476.538968,1314.604950
...,...,...,...,...,...,...,...,...,...
1543,0.321376,-0.612856,0.000395,0.239694,0.006120,0.078227,0.078228,198.148847,607.132910
1544,0.134242,-0.467288,-0.000391,0.156843,0.002944,0.054255,0.054256,138.877985,401.464649
1545,0.283167,-0.613791,0.000041,0.243243,0.005834,0.076379,0.076379,1864.865676,5938.972825
1546,-0.214313,-0.481188,0.000158,0.196728,0.004048,0.063620,0.063621,402.514525,1244.655426


In [22]:
freq_domain

,SpectralCentroid,SpectralBandwidth,PeakFrequency,TotalPower,HarmonicContent,THD
0,486.505494,217.688955,390.6250,0.000040,0.068984,0.0
1,505.692475,221.009126,585.9375,0.000041,0.004871,0.0
2,495.093217,236.112488,390.6250,0.000027,0.096089,0.0
3,508.423744,228.757150,585.9375,0.000033,0.004380,0.0
4,500.400108,272.553688,585.9375,0.000018,0.004000,0.0
...,...,...,...,...,...,...
1543,527.370639,229.188902,585.9375,0.000034,0.008538,0.0
1544,503.647678,260.977737,390.6250,0.000017,0.087189,0.0
1545,521.344495,226.701856,585.9375,0.000033,0.006336,0.0
1546,513.496838,237.097757,585.9375,0.000022,0.005161,0.0


<center><h1>Step 1: Learn Healthy Feature Statistics at 0% Load</h1></center>

<center><h1>Step 2: Expand PF State Vector for Feature Tracking</h1></center>

<p>For the PF implementation, the state vector could be:

<center>$x_i = \begin{bmatrix}
            \text{Feature 1} \\
            \text{Feature 2} \\
             ... \\
             \text{Load}
            \end{bmatrix}$</center>

You will predict each feature’s evolution over time, while also tracking the hidden load %.

<center><h1>Step 3: Define Process Model (Prediction Step)</h1></center>


<center><h1>Step 4: Define Measurement Model (Update Step)</h1></center>


<center><h1>Step 5: Resampling + State Estimation</h1></center>